In [1]:
!pip install graphrag
%pip install yfiles_jupyter_graphs --quiet 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import tiktoken
from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
import subprocess
import io
import sys
import util
from openai import OpenAI
import re
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore
from yfiles_jupyter_graphs import GraphWidget
import prompt_template


In [9]:
# concatenate all cross-reference text files
source_folder = 'documents/article_chunks'
destination_folder = 'input'
output_file = 'articles.txt'

output_file_path = os.path.join(destination_folder, output_file)
all_text = []

for filename in os.listdir(source_folder):
    if filename.endswith('.txt'):
        file_path = os.path.join(source_folder, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            all_text.append(file.read())

combined_text = '\n'.join(all_text)

with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(combined_text)



In [4]:
#remove uncommon characters 
def clean_text_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
    
    cleaned_content = ''.join(char for char in content if ord(char) < 128)
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(cleaned_content)
input_file = "input/articles.txt"
clean_text_file(input_file, input_file)


In [13]:
# Generate a Graph RAG from the provided text file. or you can run "python -m graphrag.index --root ." in terminal.
# WARNING: Running this cell may incur significant costs depending on the size of your content and the model used.

command = ['python', '-m', 'graphrag.index', '--root', '.']
result = subprocess.run(command, capture_output=True, text=True)

if result.returncode == 0:
    print("Command executed successfully.")
else:
    print("Command failed with return code", result.returncode)
    print(result.stderr)  


Command executed successfully.


In [4]:
# moved files from output/****-****
report_df = pd.read_parquet(f"output/graphs/GPT-4o-tweaked/artifacts/create_final_community_reports.parquet")
entity_df = pd.read_parquet(f"output/graphs/GPT-4o-tweaked/artifacts/create_final_nodes.parquet")
entity_embedding_df = pd.read_parquet(f"output/graphs/GPT-4o-tweaked/artifacts/create_final_entities.parquet")
relationship_df = pd.read_parquet(f"output/graphs/GPT-4o-tweaked/artifacts/create_final_relationships.parquet")
relationships = read_indexer_relationships(relationship_df)
report_df

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# Brokers and Client Management Systems\n\nThe...,1,7.5,Brokers and Client Management Systems,The impact severity rating is high due to the ...,The community is centered around brokers who m...,[{'explanation': 'Brokers are pivotal in this ...,"{\n ""title"": ""Brokers and Client Management...",b447432b-4a19-465f-af61-3096a84c6666
1,11,# NCCPL and PSX Financial Infrastructure\n\nTh...,1,8.5,NCCPL and PSX Financial Infrastructure,The impact severity rating is high due to the ...,The community is centered around the National ...,[{'explanation': 'The National Clearing Compan...,"{\n ""title"": ""NCCPL and PSX Financial Infra...",4dc88f01-ddd2-44d5-b2ff-004bcb35d532
2,12,# Trading Systems and Client Interactions\n\nT...,1,7.5,Trading Systems and Client Interactions,The impact severity rating is high due to the ...,The community is centered around trading activ...,[{'explanation': 'Trading activities are requi...,"{\n ""title"": ""Trading Systems and Client In...",3beb1142-385d-4418-8f4f-73f8b466f6c4
3,13,# The Organization and Article 16\n\nThe commu...,1,7.5,The Organization and Article 16,The impact severity rating is high due to the ...,The community is centered around The Organizat...,[{'explanation': 'The Organization plays a cru...,"{\n ""title"": ""The Organization and Article ...",d13d13ba-4cc6-4ce3-a1f0-5217bcbb4013
4,14,# Financial Trading Compliance and Infrastruct...,1,8.5,Financial Trading Compliance and Infrastructur...,The impact severity rating is high due to the ...,This community is centered around the ORGANIZA...,[{'explanation': 'The ORGANIZATION is a centra...,"{\n ""title"": ""Financial Trading Compliance ...",a3cd5a73-39d7-4f4c-87c0-6851253f5371
5,15,# Broker/Dealer and Trading Systems Community\...,1,8.5,Broker/Dealer and Trading Systems Community,The impact severity rating is high due to the ...,The community is centered around the broker/de...,[{'explanation': 'The broker/dealer is a centr...,"{\n ""title"": ""Broker/Dealer and Trading Sys...",6af78b3a-b330-4fce-b192-11aefe50a39f
6,8,# University of Alberta and ISO Standards\n\nT...,1,7.5,University of Alberta and ISO Standards,The impact severity rating is high due to the ...,The community is centered around the Universit...,[{'explanation': 'The University of Alberta is...,"{\n ""title"": ""University of Alberta and ISO...",4ca9906a-26b3-4f58-95e0-4ef5266808d9
7,9,# ISO/IEC 27001:2013 and Information Security ...,1,8.5,ISO/IEC 27001:2013 and Information Security Ma...,The impact severity rating is high due to the ...,The community is centered around the ISO/IEC 2...,[{'explanation': 'ISO/IEC 27001:2013 is an int...,"{\n ""title"": ""ISO/IEC 27001:2013 and Inform...",1d5ac032-8f43-4867-9466-298c252c33ae
8,0,# The Exchange and Financial Market Operations...,0,8.5,The Exchange and Financial Market Operations,The impact severity rating is high due to the ...,"The community is centered around The Exchange,...",[{'explanation': 'The Exchange is a central en...,"{\n ""title"": ""The Exchange and Financial Ma...",fb60783e-5a84-4cc8-8b45-be2fc344b3fd
9,1,# ISO/IEC 27001:2013 and University of Alberta...,0,7.5,ISO/IEC 27001:2013 and University of Alberta,The impact severity rating is high due to the ...,This community is centered around the ISO/IEC ...,[{'explanation': 'ISO/IEC 27001:2013 is an int...,"{\n ""title"": ""ISO/IEC 27001:2013 and Univer...",db621829-f2d4-43c3-b0e7-7a9758bf31b7


In [7]:

def convert_entities_to_dicts(df):
    """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
    nodes_dict = {}
    for _, row in df.iterrows():
        node_id = row["title"]
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
    return list(nodes_dict.values())


def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
    relationships = []
    for _, row in df.iterrows():
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships


w = GraphWidget()
w.directed = True
w.nodes = convert_entities_to_dicts(entity_df)
w.edges = convert_relationships_to_dicts(relationship_df)
w.node_label_mapping = "title"
w.col = "white" 

def community_to_color(community):
    """Map a community to a color."""
    colors = [
        "crimson",
        "darkorange",
        "indigo",
        "cornflowerblue",
        "cyan",
        "teal",
        "green",
    ]
    return (
        colors[int(community) % len(colors)] if community is not None else "lightgray"
    )


def edge_to_source_community(edge):
    """Get the community of the source node of an edge."""
    source_node = next(
        (entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community"]
    return source_node_community if source_node_community is not None else None

def edge_to_thickness(edge):
    weight = edge["properties"].get("weight", 1)  # Default to 1 if "weight" is not present
    return 1 + weight * 2  # Adjust the multiplication factor to control thickness scaling



w.node_color_mapping = lambda node: community_to_color(node["properties"]["community"])
w.edge_color_mapping = lambda edge: community_to_color(edge_to_source_community(edge))
w.node_scale_factor_mapping = lambda node: 2.5 + node["properties"]["size"] * 3 / 20
w.edge_thickness_factor_mapping = edge_to_thickness
w.circular_layout()
display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [29]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "gpt-4o-2024-08-06"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  
    max_retries=20,
)
token_encoder = tiktoken.get_encoding("cl100k_base")

reports = read_indexer_reports(report_df, entity_df, 2)
entities = read_indexer_entities(entity_df, entity_embedding_df, 2)
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,
    token_encoder=token_encoder,
)

context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 5000,
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 1000, 
    "temperature": 0.0,
}

search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=5000, 
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=False,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",
)

In [30]:
#Report utils
num_of_LLM_calls = 0
num_of_comp_violations = 0
num_of_part_violations = 0
num_of_compliances = 0
num_of_self_redundencies = 0
num_of_redundencies = 0
num_of_self_conflicts = 0
num_of_conflicts = 0

In [31]:
req_num = 41
with open(f"documents/requirements/FR/req_{req_num}.txt", mode="r", encoding="utf-8") as req_file:
    req_content = req_file.read()
    result = await search_engine.asearch(f"I want you to find all the entities that can be related to {req_content}.")
    num_of_LLM_calls += result.llm_calls



In [32]:

    
df = pd.DataFrame(result.context_data['reports'])
context_df = df.sort_values(by='occurrence weight', ascending=False) 


filtered_df = context_df[context_df['occurrence weight'] > 0.6]
result_str =  result.response
#content_context = context_df["content"][0] + "\n" + result_str

context_df.head(10)

,id,title,occurrence weight,content,rank
0,9,ISO/IEC 27001:2013 and Information Security Ma...,1.000000,# ISO/IEC 27001:2013 and Information Security ...,8.5
1,8,University of Alberta and ISO Standards,0.611111,# University of Alberta and ISO Standards\n\nT...,7.5
2,14,Financial Trading Compliance and Infrastructur...,0.555556,# Financial Trading Compliance and Infrastruct...,8.5
3,2,Information Security Management System and ISO...,0.444444,# Information Security Management System and I...,8.5
9,15,Broker/Dealer and Trading Systems Community,0.277778,# Broker/Dealer and Trading Systems Community\...,8.5
4,11,NCCPL and PSX Financial Infrastructure,0.166667,# NCCPL and PSX Financial Infrastructure\n\nTh...,8.5
10,0,The Exchange and Financial Market Operations,0.166667,# The Exchange and Financial Market Operations...,8.5
5,3,Broker/Dealer and Trading System Community,0.111111,# Broker/Dealer and Trading System Community\n...,7.5
6,12,Trading Systems and Client Interactions,0.111111,# Trading Systems and Client Interactions\n\nT...,7.5
7,13,The Organization and Article 16,0.055556,# The Organization and Article 16\n\nThe commu...,7.5


In [28]:

    
df = pd.DataFrame(result.context_data['reports'])
context_df = df.sort_values(by='occurrence weight', ascending=False) 


filtered_df = context_df[context_df['occurrence weight'] > 0.6]
result_str =  result.response
#content_context = context_df["content"][0] + "\n" + result_str

context_df.head(10)

,id,title,occurrence weight,content,rank
42,66,Organization and Information Security Management,1.000000,# Organization and Information Security Manage...,8.5
25,75,Brokerage Community and Regulatory Compliance,0.620690,# Brokerage Community and Regulatory Complianc...,7.5
26,69,Information Security Management System Community,0.551724,# Information Security Management System Commu...,8.0
33,72,User Account Management and Stock Trading Comm...,0.482759,# User Account Management and Stock Trading Co...,7.5
34,40,Trading Compliance and Customer Interaction,0.344828,# Trading Compliance and Customer Interaction\...,7.5
27,78,Information Security Community,0.275862,# Information Security Community\n\nThe Inform...,7.5
43,29,User Interaction with Database and Transaction...,0.275862,# User Interaction with Database and Transacti...,7.5
50,10,Tehran Financial Market Governance,0.172414,# Tehran Financial Market Governance\n\nThe co...,8.5
44,47,Information Processing Facilities and Security...,0.172414,# Information Processing Facilities and Securi...,8.5
1,73,User Account Management Community,0.172414,# User Account Management Community\n\nThe com...,6.5


In [33]:
sample_string = ""
for resp in result.map_responses:
    sample_string += str(resp)

pattern = re.compile(r"'answer':\s*'(.*?)',\s*'score':\s*(\d+)")
matches = pattern.findall(sample_string)

df = pd.DataFrame(matches, columns=['Answer', 'Score'])
df['Score'] = pd.to_numeric(df['Score'])
sorted_df = df.sort_values(by='Score', ascending=False) 

filtered_df = df[df['Score'] >= 0.7]
high_score_answer = '\n'.join(filtered_df['Answer'])
result_str = result_str + "\n\n" + high_score_answer

sorted_df

,Answer,Score
1,The Broker/Dealer is a central entity responsi...,90
0,"During the sale process, the user selects one ...",85
2,Brokers are pivotal in managing client account...,85
3,The Exchange collaborates with brokers and dea...,80
4,The BBO System is a crucial platform for updat...,75
5,The NCCPL provides guidelines and systems for ...,70


In [35]:
with open(f"SEARCH_OUTPUT/search_req_{req_num}.txt", mode="w", encoding="utf-8") as res_file:
    res_file.write(result_str)

In [34]:
def call_LLM(user_prompt, system_prompt):
    client = OpenAI(api_key = os.environ["GRAPHRAG_API_KEY"])
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06", 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        #max_tokens = 4096,  
        temperature = 0.2
    )
    global num_of_LLM_calls
    num_of_LLM_calls += 1
    return response


In [37]:
with open(f"SEARCH_OUTPUT/search_req_{req_num}.txt", mode="r", encoding="utf-8") as res_file:
    result_str = res_file.read() 

In [39]:
system_prompt = prompt_template.prompt_cross_reference_check(result_str,True)
with open(f"documents/requirements/FR/req_{req_num}.txt", mode = "r", encoding="utf-8") as req_file:
    req = req_file.read()
    user_prompt = prompt_template.prompt_user_compliance(req)
    result = call_LLM(user_prompt=user_prompt,system_prompt=system_prompt)
    comp_resp = result.choices[0].message.content

    with open(f"COMPLIANCE_OUTPUT/comp_res_{req_num}.txt", mode = "w", encoding="utf-8") as comp_file:
        comp_file.write(comp_resp)
        print(comp_resp)

FileNotFoundError: [Errno 2] No such file or directory: 'RAG_OUTPUT/COMPLIANCE/comp_res_41.txt'

In [42]:
with open(f"COMPLIANCE_OUTPUT/comp_res_{req_num}.txt", mode = "w", encoding="utf-8") as comp_file:
    comp_file.write(comp_resp)
    print(comp_resp)

1. CONTENT IDENTIFICATION:
- The requirement provided is related to the content in the trained document, specifically regarding the entities involved in transferring funds for selling stocks or bonds, including the customer's account, buyer's account, and brokerage account.

2. ANALYSIS:
- The analysis will focus on the relationship between the provided requirement and the content from the trained document.

3. REPORTING:

SUMMARY:
- Total related contents found: 3
- Complete violations: 1
- Partial violations: 1
- Compliant aspects: 1

DETAILED ANALYSIS:

Relationship: Compliance
Content Reference: Customer's Account, Buyer's Account, Brokerage Account
Reason: The requirement correctly describes the process of depositing the amount from sold stocks into the customer's account, withdrawing the amount from the buyer's account, and handling the brokerage fee by withdrawing it from the customer's account and depositing it into the brokerage account. This aligns with the described roles of

In [99]:
# LLM result will return compliances too. -> extract violations
def extract_violations(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    global num_of_comp_violations
    global num_of_part_violations
    global num_of_compliances
    
    violations = []
    entries = content.split('\n\n') 
    for entry in entries:
        if 'Relationship: Partial Violation' in entry:
            violations.append(entry)
            num_of_part_violations += 1
        if 'Relationship: Complete Violation' in entry:
            violations.append(entry)
            num_of_comp_violations += 1
        if 'Relationship: Compliance' in entry:
            num_of_compliances += 1
    
    return violations


In [100]:
# store extracted violations in a data frame for better analysis
data_list = [extract_violations(f"RAG_OUTPUT/COMPLIANCE/comp_res_{req_num}.txt")]
print(data_list)
data = []
def parse_item(item, counter):
    id_value = counter

    relationship_match = re.search(r'Relationship: ([^\n]+)', item)
    if relationship_match:
        relationship = relationship_match.group(1).strip()
    else:
        relationship = None

    content_reference_match = re.search(r'Content Reference: ([^\n]+)', item)
    if content_reference_match:
        content_reference = content_reference_match.group(1).strip()
    else:
        content_reference = None

    reason_match = re.search(r'Reason: ([^\n]+)', item)
    if reason_match:
        reason = reason_match.group(1).strip()
    else:
        reason = None

    suggested_solution_match = re.search(r'Suggested Solution: ([^\n]+)', item)
    if suggested_solution_match:
        suggested_solution = suggested_solution_match.group(1).strip()
    else:
        suggested_solution = None

    return {
        'ID': id_value,
        'Relationship': relationship,
        'Content Reference': content_reference,
        'Reason': reason,
        'Suggested Solution': suggested_solution
    }

counter = 1
for sublist in data_list:
    for item in sublist:
        data.append(parse_item(item, counter))
    counter += 1

df = pd.DataFrame(data, index=range(1, len(data) + 1))
df.head(10)


[['7. \n   Relationship: Partial Violation\n   Content Reference: Status of Orders\n   Reason: The requirement states that users can delete orders with the status "completed" or "canceled," but it does not clarify how the system identifies these statuses, which is crucial for effective order management.\n   Suggested Solution: Include a specification on how the system determines the status of orders eligible for deletion.']]


,ID,Relationship,Content Reference,Reason,Suggested Solution
1,1,Partial Violation,Status of Orders,The requirement states that users can delete o...,Include a specification on how the system dete...


In [101]:
# Extract suggested solutions for consistency check
#util.delete_all_files_in_directory("RAG_OUTPUT/SOLUTIONS/")
batch_size = 10
results = ""
solutions_size = 0
counter = 1
system_prompt_size = util.count_tokens(prompt_template.prompt_consistency(""), "gpt-4o")

for start in range(0, len(df), batch_size):
    end = start + batch_size
    batch = df[start:end]
    batch_str = "\n".join(f"{row.name}. {row['Suggested Solution']}" for index, row in batch.iterrows())
    solutions_size += util.count_tokens(batch_str, "gpt-4o")
    if solutions_size >= 2048 - system_prompt_size:
        with open(f"RAG_OUTPUT/SOLUTIONS/sol_{counter}.txt", 'w', encoding='utf-8') as file:
            file.write(results)
        results = ""
        solutions_size = 0
        counter += 1
    results += batch_str + "\n"
    solutions_size += util.count_tokens(batch_str, "gpt-4o")

with open(f"RAG_OUTPUT/SOLUTIONS/sol_{counter}.txt", 'w', encoding='utf-8') as file:
    file.write(results)

In [102]:
# # find redundant or contradicting solutions. (self consistency check)
# directory = 'RAG_OUTPUT/SOLUTIONS/'
# text_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
# result = ""
# if len(text_files) > 1:
#     for i in range(len(text_files)):
#         for j in range(i + 1, len(text_files)):
#             file1_path = os.path.join(directory, text_files[i])
#             file2_path = os.path.join(directory, text_files[j])
#             with open(file1_path, 'r', encoding='utf-8') as file1:
#                 file1_contents = file1.read()

#             with open(file2_path, 'r', encoding='utf-8') as file2:
#                 file2_contents = file2.read()
#             combined_contents = file1_contents + "\n" + file2_contents
#             response = call_LLM(prompt_template.prompt_user_self_consistency(combined_contents), system_prompt = prompt_template.prompt_consistency(""))
#             result += response.choices[0].message.content
# else:
#     file_path = os.path.join(directory, "sol_1.txt")
#     with open(file_path, 'r', encoding='utf-8') as file:
#         file_contents = file.read()
#         response = call_LLM(prompt_template.prompt_user_self_consistency(file_contents), system_prompt = prompt_template.prompt_consistency(""))
#         result = response.choices[0].message.content

# with open(f'RAG_OUTPUT/CONSISTENCY/SELF_CON/self_con_result_{req_num}.txt', 'w', encoding='utf-8') as confile:
#     confile.write(result)

In [103]:
# # df for self consistencies
# user_input = []
# content_from = []
# similarity_score = []
# similarity_number = []

# with open(f'RAG_OUTPUT/CONSISTENCY/SELF_CON/self_con_result_{req_num}.txt', 'r') as file:
#     lines = file.readlines()

# for line in lines:
#     if "Similarity #" in line:
#         sim_number = int(re.search(r'\d+', line).group())
#         similarity_number.append(sim_number)
#     elif "(suggested solution):" in line:
#         user_input.append(int(re.search(r'\d+', line).group()))
#     elif "Content from :" in line:
#         content_from.append(int(re.search(r'\d+', line).group()))
#     elif "Similarity Score:" in line:
#         similarity_score.append(int(re.search(r'\d+', line).group()))

# df = pd.DataFrame({
#     'Similarity Number': similarity_number,
#     'User Input (Suggested Solution)': user_input,
#     'Content From': content_from,
#     'Similarity Score': similarity_score
# })


# df.head(10)


In [104]:
# # remove redundant suggestions. conflicts are manually handled.
# content_from_list = df[df['Similarity Score'] >= 70]['Content From'].tolist()
# num_of_self_conflicts = len(df[df['Similarity Score'] == 0]['Content From'].tolist())
# num_of_self_redundencies = len(content_from_list)

# number_of_sol_files = len([f for f in os.listdir("RAG_OUTPUT/SOLUTIONS") if f.endswith('.txt')])


# for i in range(1,number_of_sol_files + 1):
#     with open(f'RAG_OUTPUT/SOLUTIONS/sol_{i}.txt', 'r') as file:
#         lines = file.readlines()

#         filtered_lines = [line for index, line in enumerate(lines, start=1) if index not in content_from_list]
#         with open(f'RAG_OUTPUT/SOLUTIONS/update/updated_sol_{i}.txt', 'w') as file:
#             file.writelines(filtered_lines)


In [105]:
# # consistency check between remaining solutions and existing requirement document.

# req_directory = 'documents/requirements/'
# sol_directory = 'RAG_OUTPUT/SOLUTIONS/update/'

# sol_files = [f for f in os.listdir(sol_directory) if f.endswith('.txt')]
# req_files = [f for f in os.listdir(req_directory) if f.endswith('.txt')]
# result = ""

# for i in range(len(sol_files)):
#     for j in range(len(req_files)):
#         sol_path = os.path.join(sol_directory, sol_files[i])
#         req_path = os.path.join(req_directory, req_files[j])
#         with open(sol_path, 'r', encoding='utf-8') as file1:
#             sol_contents = file1.read()
#         with open(req_path, 'r', encoding='utf-8') as file2:
#             req_contents = file2.read()
#         response = call_LLM(prompt_template.prompt_user_consistency(sol_contents), system_prompt = prompt_template.prompt_consistency(req_contents))
#         result += response.choices[0].message.content

# with open(f"RAG_OUTPUT/CONSISTENCY/con_res_{req_num}.txt", 'w', encoding='utf-8') as confile:
#     confile.write(result)


In [106]:
# # df for consistencies
# user_input = []
# content_from = []
# similarity_score = []
# similarity_number = []

# with open(f"RAG_OUTPUT/CONSISTENCY/con_res_{req_num}.txt", 'r') as file:
#     lines = file.readlines()

# for line in lines:
#     if "Similarity #" in line or "Contradiction #" in line:
#         sim_number = int(re.search(r'\d+', line).group())
#         similarity_number.append(sim_number)
#     elif "(suggested solution):" in line:
#         user_input.append(int(re.search(r'\d+', line).group()))
#     elif "Content from " in line:
#         content_from.append(int(re.search(r'\d+', line).group()))
#     elif "Similarity Score:" in line:
#         similarity_score.append(int(re.search(r'\d+', line).group()))
# df = pd.DataFrame({
#     'Similarity Number': similarity_number,
#     'User Input (Suggested Solution)': user_input,
#     'Content From': content_from,
#     'Similarity Score': similarity_score
# })

# content_from_list = df[df['Similarity Score'] >= 70]['Content From'].tolist()
# num_of_conflicts = len(df[df['Similarity Score'] == 0]['Content From'].tolist()) - num_of_self_conflicts
# num_of_redundencies = len(content_from_list) - num_of_self_redundencies

# df.head(10)


In [107]:
print("Number of LLM calls: ",num_of_LLM_calls)
print("Number of complete violations: ", num_of_comp_violations)
print("Number of partial violations: ", num_of_part_violations)
print("Number of compliances: ", num_of_compliances)

Number of LLM calls:  8
Number of complete violations:  0
Number of partial violations:  1
Number of compliances:  7


In [108]:
res_df = pd.read_csv("RAG_OUTPUT/result.csv")
res_df.loc[len(res_df)] = [req_num,num_of_LLM_calls,num_of_part_violations,num_of_comp_violations,num_of_compliances]


In [109]:
res_df.to_csv("RAG_OUTPUT/result.csv",index = False)